# Numerical Panel Methods 🚢 

A panel method is a numerical approach to constructing potential flows around engineering shapes such as an offshore platform, sail or ship hull. Panel methods are typically fast enough for design iteration on 3D geometries, but they come with modelling and numerical errors that need to be quantified. 

A panel method is a type of Boundary Element Methods (BEM), which have a rich and somewhat intimidating mathematical background. However, the mathematics and numerics of panels methods are very straightforward and give a good introduction to numerical analysis as well as being useful in application. 

## Method overview

The basic steps of a potential flow panel method can be summarized as:
 - The boundaries of the fluid domain are given as an input and covered in small panels, each with their own influence potential $\phi$.
 - The total potential $\varphi$ defined as the superposition of each panel's $\phi$ scaled by an unknown strength $q$.
 - The strengths are determined such that the flow boundary conditions, such as the normal velocity condition $U_n=\frac{\partial\varphi}{\partial n}$, are satisfied on every panel.
 - With the potential now defined, any quantity of interest can be measured, such as the surface pressure $p$ using $\vec u=\vec\nabla\varphi$ and the Bernoulli equation.

We will developing a programme to follow these steps, and will raise (and address) a few important numerical questions in the process:
 1. _How can we calculate the influence of each panel $\phi$?_ **Gaussian quadratures.**
 2. _How should we compute the derivatives of $\phi$ and other functions?_ **Automatic differentiation.**
 3. _How can we find the optimal $q$ for each panel?_ **Set-up and solve a linear system.**
 4. _How should we determine if a method is working?_ **Convergence and validation tests.** 

## Potential of a source panel 🚰

The velocity potential of a panel per unit strength (called the _influence_) is defined as

$$ \phi(\vec x) = \int\int G(\vec x,\vec S(\xi,\zeta))\ d\xi d\zeta $$

where $S$ is the surface of the panel, described by coordinates $\xi$ and $\zeta$, and $G$ is the Green Function, which needs to be a solution of the laplace equations $\nabla^2 G=0$. For every point $\vec x$ where you want the potential, you must evaluate this double integral over the surface coordinates.

The functions inside this integral are known: the geometry $S$ is _the_ input (a portion of a sphere, a bit of the hull defined by a spline, etc), and we will choose $G$ to be the source potential

$$ G(\vec x, \vec c) = \frac{-1}{|\vec x-\vec c|}$$

Despite this, the integral for $\phi$ is not analytically solvable, except for special-case $S$ shapes. This is in stark contrast to derivatives! If we had $\phi=\frac{\partial}{\partial \xi} G(x,S(\xi,\zeta))$ we would have no problem carrying out this derivative using the chain rule, even for complex shapes. Integrals don't have an equivalent rule that always works! Our first question is therefore:

 1. _How can we calculate the influence of each panel $\phi$?_

## Numerical integration quadratures

While we can't calculate the influence analytically, **quadratures** are simple and effective methods to numerically approximate integrals. Consider the integral

$$ F = \int_{-h/2}^{h/2} f(x) dx $$

where $h$ is a small interval. You're already familiar with many example quadratures we can use to estimate this integral:
 - mid-point rule: $F \approx h f(0)$
 - trapezoid rule: $F \approx \frac h2 [f(-h/2)+f(h/2)]$
 - Simpson's rule: $F \approx \frac h6 [f(-h/2)+4f(0)+f(h/2)]$

These algorithms are all simple but they require 1,2 and 3 function evaluations, respectively. How much more accuracy are we getting for the increased amount of work? We can determine this by expressing $f$ in terms of it's Taylor series

$$ f(x) = f_0+\frac x2 f_0' + \frac{x^2}6 f_0'' + \frac{x^3}{12} f_0''' + \ldots $$

where subscript 0 denotes evaluation at $x=0$ and each tick $'$ indicates a derivative. Substitution into the integral and algebraic manipulation gives:
 - mid-point: $F = hf_0 + \frac 1{24} h^3 f_0'' + O(h^4)$
 - trapezoid: $F = \frac h2 [f(-h/2)+f(h/2)] -\frac 1{12}h^3 f_0''+O(h^4)$
 - Simpson's: $F = \frac h6 [f(-h/2)+4f(0)+f(h/2)] + O(h^4)$

where $O(h^4)$ means all the following terms are proportional to $h$ to the power of 4 or greater. Since the quadrature neglected these additional terms, they have a well-defined _truncation error_.

<div class="alert alert-success">
<b> Type 3: Numerical Truncation Error </b>

Errors due to truncating an infinite series at some power of the step length $h$
</div>

Unlike system description errors or modelling errors, numerical truncation errors can be well estimated and controlled without access to the "true" result. We can see that the simple midpoint rule is quite accurate: **as long as the function is smooth such that the derivatives don't blow up, a small value of $h$ will make $h^3 f''$ negligible**. Surprisingly, the trapezoid rule _doubles_ the error of the midpoint rule and doubles evaluations, so it's not a good choice.

The midpoint rule applied to the influence potential integral gives:

$$ \phi(\vec x) \approx A G(\vec x,\vec c)=\frac{-A}{|\vec x-\vec c|} $$

where $A$ and $\vec c$ are the area and centroid of the panel. Note that is just a point-source located at the panel centroid with $s=A$. This should raise some red flags because the whole point of using panels was to smooth out the singularity, and now it's back! 

_Will this approximation really be sufficiently accurate for our panel method?_

### Julia coding example

Let's answer this question using an example: a 2D point source at the origin $G=\frac 12 \log(x^2)$ integrated on panels covering the x axis.

⚠️ WARNING ⚠️: This part of the course uses Julia, not Python or MATLAB. Julia has a bunch of advantages that help when writing a panel method. It's simple and flexible and [very fast](https://julialang.org/benchmarks/) at numerical computing!

![Julia benchmarks](https://julialang.org/assets/images/benchmarks.svg)

The syntax is similar to Python and MATLAB and here is a list of [note-worthy differences between Julia and other languages](https://web.mit.edu/julia_v0.6.2/julia/share/doc/julia/html/en/manual/noteworthy-differences.html) to help you make the switch. The code below is a good example of basic Julia syntax, and I've added extra comments to explain each step. 

In [ ]:
# First define the functions we need
### Look how easy this is in Julia! No more `def G(x): return stuff`
G(x) = 0.5log(x^2)                 # 2D source at x₀=0
G_int(x) = 0.5x*log(x^2)-x         # indefinite integral
midpoint(f,a,b) = (b-a)*f((a+b)/2) # Midpoint rule for any function `f`

# Next let's plot the Green's function
### Plots is the basic plotting package for Julia
using Plots 
plot(range(-1,1,50),G,label="G",ylims=(-1.5,0),xlabel="x")

# Split up the x-axis into panels
### The range function and slicing is like Python
### but adding vectors works automatically
x = range(-1,1,10) # "panels" running along x-axis
scatter!(x,zero,shape=:vline,c=:black,label="panels")
a = x[1:end-1]     # lower limits of each panel
b = x[2:end]       # upper limits
m = (a+b)/2        # panel midpoint

# Finally, we can compare the exact and numeric panel integrals
### The `f.(x)` syntax "broadcasts" the function over input vectors
### similar to a list comprehension.
exact = G_int.(b) - G_int.(a) # same as [G_int(aᵢ)-G_int(bᵢ) for (aᵢ,bᵢ) in zip(a,b)]
numeric = midpoint.(G,a,b)    # same as [midpoint(G,aᵢ,bᵢ) for (aᵢ,bᵢ) in zip(a,b)]
scatter!(m,exact,label="Exact integral")
scatter!(m,numeric,alpha=0.5,label="Midpoint rule")

The exact integral and the midpoint rule results are directly on top of each other at the edges of the range. However, $G_0=-\infty$ making the midpoint rule singular despite the fact that the true integral is well defined.

So the midpoint rule is sufficient as long as we are not too close to the singularity, i.e. $|x|>>h$. However, we need a better approximation near 0. 

## Gaussian quadratures

[Gaussian quadratures](https://en.wikipedia.org/wiki/Gaussian_quadrature) are family of simple but **very** accurate integration rules which are used in many engineering methods such as finite element methods. A Gaussian quadrature requires evaluating the function at $n$ special points $x_i$ (instead of simple fractions of $h$) and computing their sum with weights $w_i$.

 - Gauss quadrature rule: $F = \int_{-1}^{1} f(x) dx \approx \sum_{i=1}^n w_i f(x_i)$

The 1-point rule uses $x=[0],\ w=[2]$ matching the midpoint rule. However, the error is proportional to $h^{2n+1}f^{(2n)}$, so every additional point scales the error down by $h^2$ for smooth functions! This is **much** better than the fractional-step methods like Trapezoid and Simpson's rule.

![2-point Gauss quadrature](https://upload.wikimedia.org/wikipedia/commons/9/93/Comparison_Gaussquad_trapezoidal.svg)

As illustrated in this figure, the 2-point quadrature uses $x=[\frac{-1}{\sqrt{3}},\frac{1}{\sqrt{3}}]$ with $w=[1,1]$ and exactly integrates any cubic (since $f^{(4)}=0$ for a cubic). 

Let's try this on our example problem:

In [ ]:
# Define the Gauss-Legendre points and weights
### `w'` is the transpose of the weight vector, 
### and `w'*v = w₁v₁+w₂v₂+...` is the inner product
xgl,wgl = [-1/√3,1/√3],[1,1]
quadgl(f;x=xgl,w=wgl) = w'*f.(x) # same as sum(wᵢ*f(xᵢ) for (xᵢ,wᵢ) in zip(x,w))

# Convert the range from [-1,1] to [a,b]
### This gives an example of how multi-line functions are defined
### as well as in-place functions like `x->g(2x)`
function quadgl_ab(f,a,b)
    h,j = (b-a)/2,(a+b)/2 # spacing and center
    h*quadgl(t->f(j+h*t))
end

# Apply to the example
gauss = quadgl_ab.(G,a,b)
plot(range(-1,1,50),G,label="G",ylims=(-1.5,0),xlabel="x")
scatter!(x,zero,shape=:vline,c=:black,label="panels")
scatter!(m,exact,label="Exact integral")
scatter!(m,numeric,alpha=0.5,label="Midpoint rule")
scatter!(m,gauss,alpha=0.5,label="2-point Gauss")

The 2-point Gaussian quadrature requires two function evaluations, but gives nearly perfect results for all  $x\ne 0$. The estimate on the center panel is finite (yay!) and it's error is proportional to $h$. 

Based on this, our simple panel method will use the 1-point (midpoint) quadrature for all $|\vec x-\vec c|^2 \gg A^2$, and a 2-point quadrature in both $\xi$ and $\zeta$ (requiring 2x2=4 function evaluations) otherwise.

## Velocity influence

Now that the panel influence potential is defined, it's influence on the velocity is $\vec \nabla \phi$. We will need to compute these velocities both in setting the boundary conditions and in evaluating the solution (once we have it), leading to our second numerical question and some optional solutions...

 2. _How should we approximate these derivatives numerically?_
  - Try to hand-calculate the derivatives and code them without messing up
  - Use finite differences and accept surprisingly big numerical errors
  - Let the computer determine the analytic derivatives for us

The third option sounds magical at first, but the derivative rules are very simple algorithms like: 

$$ (f(g(x)))' = f'(g(x)) g'(x) $$

Letting the computer apply these rules for you is called [automatic differentiation](https://www.ams.org/publicoutreach/feature-column/fc-2017-12) or AutoDiff. Julia *is* fast, but a huge reason to use it is that **all Julia code is differentiable**. 

Let's test out all three options on our 2D source example:

In [ ]:
using ForwardDiff: derivative
dG_hand(x) = 1/x # (0.5log(x²))' = 0.5log'(x²)*(x²)' = 0.5/x²*2x = 1/x
dG_finite(x;e=eps()) = (G(x+e)-G(x-e))/2e ### eps() is the smallest floating point number
dG_auto(x) = derivative(G,x) # Wow! Hard work!!
plot(range(-1,1,50),dG_hand,xlabel="x",label="By hand",ylims=(-6,6))
scatter!(range(-1,1,12),dG_finite,ls=:dashdot,label="FiniteDiff")
scatter!(range(-1,1,12),dG_auto,ls=:dash,alpha=0.5,label="AutoDiff")

The finite difference results are surprisingly bad! We've used the smallest step possible, but we have not converged to the analytic derivative produced by the other two methods. This is a second type of numerical error:

<div class="alert alert-success">
<b> Type 4: Numerical Floating Point Error </b>

Errors due to representing real numbers with a finite precision binary number
</div>

While a [double-precision number](https://en.wikipedia.org/wiki/Floating-point_arithmetic) (the Julia default) is typically fine, finite differences over small intervals become unstable due to rounding.

In contrast, AutoDiff generates functions **for the analytic derivatives!** And doing so automatically avoids potential blunders in our hand calculation and bugs in our implementation. Clearly, we'll use AutoDiff in this solver.

## Total flow equations

With the influence integral and it's derivatives defined, we can finally start working on solving for the complete flow around a set of panels.

The potential of the full surface is simply the superposition of each panel's contribution,

$$ \varphi(x) = \sum_{i=1}^N q_i \phi_i(x) $$

where $N$ is the number of panels, $q$ is the vector of **unknown** panel strengths and $\phi_i$ is influence of panel $i$.

Since we used a $G$ satisfying $\nabla^2 G=0$ and this equation is linear, _any_ vector $q$ will satisfy the laplace equation $\nabla^2\varphi=0$ and be a valid potential flow. This is nice since we can't mess that up, but it means we need an additional equation to determine the _correct_ $q$ for a given geometry and flow condition.

 3. _How can we determine the correct $q$ for each panel?_ 

## Apply boundary conditions

The additional equations from our problem description are the boundary conditions. Defining $\vec U$ as the free stream velocity and $\hat n$ as the surface normal, the conditions in an infinite fluid (no free surface) are
 - Flow tangency on the solid body's surface: $U_n+u_n = U_n+\frac{\partial\varphi}{\partial n}=0$
 - No disturbance far from the body: $u(\infty)\rightarrow 0$ 

The second condition is achieved automatically since $u(r) \sim \frac{\partial G}{\partial r} = 1/r^2$. Therefore, the first condition must be used to set $q$.

Substituting the equation for $\phi$ into the body BC, we have

$$ \vec U \cdot \hat n(\vec S) + \sum_{i=1}^N \frac{\partial\phi_i}{\partial n}(\vec S) q_i = 0$$

This boundary condition is linear in $q$ and applies to every point on the body surface. Applying the boundary condition at $N$ specific locations will create $N$ linear equations of the $N$ unknown components of $q$. We will choose the centroid $\vec c$ of each panel to apply this condition. Defining 

$$ a_{i,j} = \frac{\partial\phi_i}{\partial n}(\vec c_j), \quad b_j = -\vec U \cdot \hat n(\vec c_j) $$

as the component of the influence matrix $A$ and the excitation vector $b$, we have

$$\sum_{i=1}^N a_{i,j} q_i = b_j \quad j=1\ldots N$$

or simply $Aq = b$. So, _how can we determine the correct $q$ for each panel?_ Construct $A,b$ and type

```julia
q = A\b
```

## 3D example: sphere

Let's illustrate this with the 3D example of the flow around a sphere.

The `sphere` function below creates a vector of panels using the parametric equation equation 

$$ [S_x,S_y,S_z] = [a\cos(θ₂)*\sin(θ₁),a\sin(θ₂)*\sin(θ₁),a\cos(θ₁)] $$

where `θ₁,θ₂` are the azimuth and polar angles.

In [ ]:
include("../src/solve.jl")
using Plots
"""
    sphere(h;a=1) -> Table(panel_props)

Sample a sphere of radius `a` such that the arc-length ≈ h in each direction.
θ₁=[0,π]: azimuth angle, θ₂=[0,2π]: polar angle.
"""
function sphere(h;a=1)
    S(θ₁,θ₂) = a .* [cos(θ₂)*sin(θ₁),sin(θ₂)*sin(θ₁),cos(θ₁)]
    dθ₁ = π/round(π*a/h)
    mapreduce(vcat,0.5dθ₁:dθ₁:π) do θ₁
        dθ₂ = 2π/round(2π*a*sin(θ₁)/h) # get polar step at this azimuth
        param_props.(S,θ₁,0.5dθ₂:dθ₂:2π,dθ₁,dθ₂)
    end |> Table
end

# Create the panels and check how many of them were made
h = 0.3; panels = sphere(h); N=length(panels)

# Get the vector of centroids (`panels.x`) and split that into  `x,y,z` vectors
x,y,z = eachrow(stack(panels.x));
plot(x,y,z,legend=false,title="Sphere represented with $N panels")

The function sets the azimuth and polar spacings so the arc length on each panel is nearly `h`. Then, it evaluates the function `S(θ₁,θ₂)` at those spacings, filling a vector with all the panel information using the `param_props` function defined in `solve.jl`. 

Each panel saves it's coordinate vector `x` (plotted above), area `dA`, and unit normal vector `n`. 

I've set $h=0.3$ and left the $a=1$ default, which creates approximately 4πa²/h²=139 panels. This shows that $N \sim 1/h^2$, which we might get worried about in a minute...

Let's check if the area's add up to 4π and they are all $\approx h^2$

In [ ]:
A_error = sum(panels.dA)/4π-1
A_percent = round(100A_error,digits=1)
plot(z,panels.dA/h^2,ylim=(0,2),xlabel="z",ylabel="dA/h²",
    legend=false, title="Error in total surface area=$A_percent%")

Looks good!